## Pair Programming ETL Carga I

In [1]:
import pandas as pd
import mysql.connector

Cread la BBDD con el nombre de energía. 

La BBDD tendrá la siguiente forma:

Cread las tablas de la BBDD:

- Tabla fechas
- Tabla nacional_renovable_no_renovable
- Tabla comunidades_renovable_no_renovable
- Tabla comunidades

Al final nuestra BBDD tendrá el siguiente aspecto:

In [ ]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab", # aquí tendréis que poner vuestra contraseña de MySQL
      auth_plugin = 'mysql_native_password') # esta línea no es necesaria solo lo tendréis que incluir si os sale el siguiente error: Authentication plugin 'caching_sha2_password' is not supported
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
crear_bbdd('energía')

In [ ]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}",  
                                     auth_plugin = 'mysql_native_password')
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
#insertas de sql

In [ ]:
df_nacional = pd.read_csv('../datos/df_energia_nacional.csv', index_col = 0)
df_ccaa = pd.read_csv('../datos/df_ccaa_clean.csv', index_col = 0)
df_comunidades = pd.read_csv('../datos/poblacion_comunidades_clean.csv', index_col = 0)

BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [ ]:
def check_comunidades(contraseña, nombre_bbdd):
    
    cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
    mycursor = cnx.cursor()


    query_existe_comunidad = f"""
            SELECT DISTINCT comunidad FROM comunidades
            """
    mycursor.execute(query_existe_comunidad)
    comunidades = mycursor.fetchall()
    return comunidades

In [ ]:
check_comunidades("AlumnaAdalab", "energía")

In [ ]:
for indice, fila in df_comunidades.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidades = f"""
                INSERT INTO comunidades (comunidad) 
                VALUES ( "{fila["Comunidad"]}");
                """  
    
    comunidades = check_comunidades("AlumnaAdalab", "energía")
    
    if len(comunidades) == 0 or fila['Comunidad'] not in comunidades[0]: 
        crear_insertar_tabla('energía', 'AlumnaAdalab', query_comunidades)

    else:
        print(f"{fila['Comunidad']} ya esta en nuestra BBDD") 